In [45]:
#Import pandas, matplotlib.pyplot, and seaborn
import pandas as pd
import numpy as np
import os
import statsmodels as sm
from statsmodels.tsa import seasonal
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import seaborn  as sns

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns # For all our visualization needs.
import statsmodels.api as sm # What does this do? Find out and type here.
from statsmodels.graphics.api import abline_plot # What does this do? Find out and type here.
from sklearn.metrics import mean_squared_error, r2_score # What does this do? Find out and type here.
from sklearn.model_selection import train_test_split #  What does this do? Find out and type here.
from sklearn import linear_model, preprocessing # What does this do? Find out and type here.
import warnings # For handling error messages.
# Don't worry about the following two instructions: they just suppress warnings that could occur later. 
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
import os

In [2]:
myfold = os.getcwd()

In [3]:
df_eng = pd.read_csv(myfold + '/takehome_user_engagement.csv')


In [4]:
df_user = pd.read_csv(myfold + '/takehome_users.csv', encoding='latin-1')
df_user = df_user.rename({"object_id":"user_id"}, axis=1)

In [5]:
df_user.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [6]:
df_eng.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [7]:
df_eng['time_stamp'] = pd.to_datetime(
    df_eng['time_stamp'], format='%Y-%m-%d %H:%M:%S'
)

In [8]:
# defining an adopted user
df_agg = df_eng.set_index('time_stamp')
adopted = []
users= df_agg.user_id.unique()

for i in users:
    id_filter = df_agg["user_id"] == i
    filter = df_agg[id_filter].resample("1D").count()
    filter = filter.rolling(window=7).sum()
    filter = filter.dropna()
    adopted.append(any(filter["visited"].values >= 7))

In [9]:
# applying 'adopted_user' logic onto df #
user_adoption = list(zip(users, adopted))

df_adopted = pd.DataFrame(user_adoption)
df_adopted.columns = ["user_id", "adopted_user"]
df = df_user.merge(df_adopted, on="user_id", how="left")
    

In [10]:
# mapping 'adopted_user' #
df_adopted['adopted_user'] = df_adopted['adopted_user'].map({False:0, True:1, np.nan:0})
df.dropna(subset=["adopted_user"], inplace=True)
df["adopted_user"] = df["adopted_user"].astype(int)

In [11]:
# mapping 'invited_by_user' 
df["invited_by_user"] = df["invited_by_user_id"].map(lambda x: 0 if np.isnan(x) else 1)

In [12]:
# final df #
df_final = df[["adopted_user", "invited_by_user", "creation_source", \
         "opted_in_to_mailing_list", "enabled_for_marketing_drip"]]

In [14]:
# make dummy variable for creation source
df_model = pd.get_dummies(df_final, columns = ['creation_source'], drop_first=True)

In [28]:
y = df_model['adopted_user']
X = df_model.drop('adopted_user', axis=1)

In [29]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, random_state=42)

### Model

We will build a quick random forest model to explore feature importance. In this case a random forest model will provide a good approach as there are only few factors for which a user decides to adopt the product.

In [49]:
# use random forest model 

clf = RandomForestClassifier(random_state=42)
# use CV to tune number of trees
params = {'n_estimators': [10, 50, 100, 200]}
model = GridSearchCV(clf, params)
model.fit(Xtrain, ytrain)
print('Best parameters were', model.best_params_)
print('Best average precision was', model.best_score_)

Best parameters were {'n_estimators': 10}
Best average precision was 0.9335046002288125


In [31]:
# evluation of the model
from sklearn.metrics import classification_report, accuracy_score
model = RandomForestClassifier(
    random_state=42, class_weight='balanced', n_estimators=100
)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)
print('Accuracy of test set was', accuracy_score(ytest, ypred))
print(classification_report(ytest, ypred))

Accuracy of test set was 0.6097008159564823
              precision    recall  f1-score   support

           0       0.92      0.63      0.75      2025
           1       0.09      0.41      0.15       181

    accuracy                           0.61      2206
   macro avg       0.51      0.52      0.45      2206
weighted avg       0.85      0.61      0.70      2206



In [19]:
# take a look at feature importance
imp = pd.Series(model.feature_importances_, index=X.columns)
imp = imp.sort_values(ascending=False)
print(imp)

opted_in_to_mailing_list              0.257079
enabled_for_marketing_drip            0.250232
creation_source_ORG_INVITE            0.131173
invited_by_user                       0.110504
creation_source_SIGNUP                0.103836
creation_source_PERSONAL_PROJECTS     0.081274
creation_source_SIGNUP_GOOGLE_AUTH    0.065902
dtype: float64


### Conclusion

**With just a brief data exploration, the top factors predicting user adoption are:**

* opted_in_to_mailing_list              
* enabled_for_marketing_drip            
* creation_source_ORG_INVITE          
* invited_by_user 